In [1]:
# Install Hugging Face libraries
!pip install  --upgrade datasets==3.3.2 accelerate==1.4.0 bitsandbytes==0.45.3 pillow protobuf sentencepiece
# !pip install ultralytics
# !pip install evaluate nltk rouge-score
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
!pip install fastapi uvicorn pyngrok nest-asyncio

In [ ]:
from huggingface_hub import login
login()

In [4]:
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from skimage.io import imread
from skimage.transform import resize


# Parameters
IMAGE_SIZE = (256, 256)  # You can change this
class_labels = ['Fake', 'Real']

In [5]:
# ...existing imports...
from datasets import load_dataset, DatasetDict
from PIL import Image
import os

# Few-shot examples for the assistant
few_shot_examples = [
    {
        "DocumentType": "Bank Statement",
        "Authenticity": "Fake",
        "FraudScore": 0.92,
        "Reason": "Suspicious transaction entries and mismatched dates."
    },
    {
        "DocumentType": "Salary Slip",
        "Authenticity": "Real",
        "FraudScore": 0.05,
        "Reason": "Consistent formatting and verified employer details."
    },
    {
        "DocumentType": "ID Card",
        "Authenticity": "Fake",
        "FraudScore": 0.88,
        "Reason": "Font inconsistencies and unnatural linguistic patterns."
    },
    {
        "DocumentType": "ID Card",
        "Authenticity": "Fake",
        "FraudScore": 0.95,
        "Reason": "Face Replacement and morphing"
    }
]

# System message for the assistant
system_message = """Your task is to:
    - Identify the **document type**.
    - Determine whether the document is **Real** or **Fake** based on below reasoning:
    - Suspicious or inconsistent entries.
    - Morphing and replacement indicators
    - Font inconsistencies.
    - Violations of standard banking or accounting practices.
    - Textual or numeric manipulation (e.g., formatting issues, overwritten values).
    - Metadata mismatches (e.g., conflicting dates, fake signatures/stamps).
    - Unnatural linguistic patterns or overly generic phrasing.
    - Semantic inconsistencies or hallucinated data.

Return your output in the following json format:
DocumentType: <e.g., Bank Statement, Salary Slip, ID Card>
Authenticity: <Original, Fraud, Real, Fake, Genuine>
FraudScore: <0.0 to 1.0, where 1.0 is highly likely fraud>
Reason: <Clear, concise explanation with observed issues related to authenticity>
"""

# Few-shot prompt builder
def build_fewshot_prompt():
    prompt = system_message + "/n/n"
    prompt += "Examples:/n"
    for ex in few_shot_examples:
        prompt += f"{ex}/n"
    prompt += "/nNow analyze the following document:"
    return prompt

def format_data(img):
    prompt = build_fewshot_prompt()
    return [
        {
            "role": "system",
            "content": [{"type": "text", "text": prompt}],
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": img,
                },
                {
                    "type": "text",
                    "text": "Identify the documentType, authenticity, fraud score, and reason.",
                },
            ],
        }
    ]


In [6]:
import torch
from datasets import load_dataset
from transformers import AutoProcessor, AutoModelForImageTextToText, BitsAndBytesConfig

# # Load Model with PEFT adapter and 4-bit quantization for evaluation
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_storage=torch.bfloat16,
)

# pip install accelerate

from transformers import AutoProcessor, Gemma3ForConditionalGeneration
from PIL import Image
import requests
import torch

model_id = "google/gemma-3-4b-it"

model = Gemma3ForConditionalGeneration.from_pretrained(
    model_id, device_map="auto"
).eval()

processor = AutoProcessor.from_pretrained(model_id)

# Initialize VLM components
ID_model = AutoModelForImageTextToText.from_pretrained("AliceRolan/gemma-idcard-FT",
    device_map="auto",
    torch_dtype=torch.bfloat16,
    attn_implementation="eager",
    quantization_config=bnb_config,
)
ID_processor = AutoProcessor.from_pretrained("AliceRolan/gemma-idcard-FT")

# Initialize VLM components
currency_model = AutoModelForImageTextToText.from_pretrained("AliceRolan/gemma-currency-FT",
    device_map="auto",
    torch_dtype=torch.bfloat16,
    attn_implementation="eager",
    quantization_config=bnb_config,
)
currency_processor = AutoProcessor.from_pretrained("AliceRolan/gemma-currency-FT")

def execute_prompt(model, processor, img,top_p,temperature, category=None):
    if category is not None:
        messages = [
          {
              "role": "system",
              "content": [{"type": "text", "text": f"You are a helpful banking assistant and are a forensic financial analyst specializing in detecting fraud, forgery, and AI-generated content in banking documents.{system_message}"}]
          },
          {
              "role": "user",
              "content": [
                  {"type": "image", "image": img},
                  {"type": "text", "text": f"Idenitify the documentType. it is identified as {category} document.Analyze provide reason/explaination for provided authenticity as explanation and provide fraud score as fraudScore and prediction confidence score as confidenceScore. Return your output in json format"}
              ]
          }
        ]
    else:
        messages = format_data(img)
    inputs = processor.apply_chat_template(
        messages, add_generation_prompt=True, tokenize=True,
        return_dict=True, return_tensors="pt"
    ).to(model.device, dtype=torch.bfloat16)

    input_len = inputs["input_ids"].shape[-1]

    with torch.inference_mode():
        generation = model.generate(**inputs, max_new_tokens=1024,
                                    do_sample=True,top_p=top_p, temperature=temperature,)
        generation = generation[0][input_len:]

    decoded = processor.decode(generation, skip_special_tokens=True)
    return decoded




AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
# Install the necessary libraries
# !pip install "fastapi[all]" Pillow

from fastapi import FastAPI, UploadFile, File
from fastapi.middleware.cors import CORSMiddleware
from PIL import Image
import io
import base64
import io
import json
from fastapi import FastAPI, HTTPException
from fastapi.responses import JSONResponse
from pydantic import BaseModel
from PIL import Image
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)
import json
import time
@app.get('/')
async def root():
    """A simple root endpoint to confirm the server is running."""
    return {'hello'}
import traceback
class PredictRequest(BaseModel):
    """
    Pydantic model to define the request body structure.
    It expects a 'category' string and a 'base64_image' string.
    """
    docType: str
    category: str
    base64_image: str


@app.post('/trainpredict')
def vlm_predict_base64(request: PredictRequest):
    """
    An endpoint to handle a Base64-encoded image for a visual language model.
    The Base64 string is decoded into a PIL Image object.
    """
    try:
        # Decode the Base64 string to bytes
        # The string might have a prefix like "data:image/png;base64," which needs to be removed.
        header, encoded_string = request.base64_image.split(',', 1) if ',' in request.base64_image else ('', request.base64_image)
        file_content = base64.b64decode(encoded_string)

        # Open the content as a PIL Image object from a byte stream
        img = Image.open(io.BytesIO(file_content))
        start_time = time.time()
        # d. VLM prediction
        if "currency" in request.docType.lower():
          ft_vlm_raw_output = execute_prompt(currency_model, currency_processor, img, 0.9, 0.5)
        else:
          ft_vlm_raw_output = execute_prompt(ID_model, ID_processor, img, 0.9, 0.5)

        import json

        # Extract the inner value string
        # print("fl:",ft_vlm_raw_output)
        # inner_value_str = ft_vlm_raw_output.split('": "')[1].strip('"\n,')

        # Split the inner string by ', ' to get key-value pairs
        pairs = ft_vlm_raw_output.split(', ')

        # Create a dictionary for the inner values
        inner_dict = {}
        for pair in pairs:
            # Split each pair by ': '
            key, value = pair.split(': ', 1)
            # Check for FraudScore to convert to float
            if key == 'FraudScore':
                inner_dict[key] = float(value)
            else:
                inner_dict[key] = value

        # Create the final nested dictionary
        # result_dict = {"finetuned_vlm_prediction": inner_dict}

        # Convert the dictionary to a JSON string
        json_output = json.dumps(inner_dict)


        return JSONResponse(content={"finetuned_vlm_prediction": inner_dict,
                                      "finetuned_vlm_fraud_score": inner_dict.get("FraudScore", "")})



    except HTTPException as http_exc:
        raise http_exc
    except Exception as e:
        print(f"An error occurred: {e}")
        traceback.print_exc()
        return JSONResponse(status_code=500, content={"error": f"An unexpected error occurred: {e}"})


@app.post('/predict')
def vlm_predict_base64(request: PredictRequest):
    """
    An endpoint to handle a Base64-encoded image for a visual language model.
    The Base64 string is decoded into a PIL Image object.
    """
    try:
        # Decode the Base64 string to bytes
        # The string might have a prefix like "data:image/png;base64," which needs to be removed.
        header, encoded_string = request.base64_image.split(',', 1) if ',' in request.base64_image else ('', request.base64_image)
        file_content = base64.b64decode(encoded_string)

        # Open the content as a PIL Image object from a byte stream
        img = Image.open(io.BytesIO(file_content))
        start_time = time.time()
        # d. VLM prediction
        if "currency" in request.docType.lower():
          ft_vlm_raw_output = execute_prompt(currency_model, currency_processor, img, 0.9, 0.5)
        else:
          ft_vlm_raw_output = execute_prompt(ID_model, ID_processor, img, 0.9, 0.5)

        import json

        # Extract the inner value string
        # print("fl:",ft_vlm_raw_output)
        # inner_value_str = ft_vlm_raw_output.split('": "')[1].strip('"\n,')

        # Split the inner string by ', ' to get key-value pairs
        pairs = ft_vlm_raw_output.split(', ')

        # Create a dictionary for the inner values
        inner_dict = {}
        for pair in pairs:
            # Split each pair by ': '
            key, value = pair.split(': ', 1)
            # Check for FraudScore to convert to float
            if key == 'FraudScore':
                inner_dict[key] = float(value)
            elif key == 'DocumentType':
                continue
            else:
                inner_dict[key] = value

        # Create the final nested dictionary
        # result_dict = {"finetuned_vlm_prediction": inner_dict}

        # Convert the dictionary to a JSON string
        json_output = json.dumps(inner_dict)

        time_taken_ft = time.time() - start_time
        start_time = time.time()
        vlm_raw_output = execute_prompt(model, processor, img, 0.9, 0.5,category=request.category)

        # Extract the JSON part from the VLM's raw output
        try:
            vlm_parsed_output = json.loads(vlm_raw_output.split("json")[1].replace('`','').strip())
        except (json.JSONDecodeError, IndexError):
            raise HTTPException(status_code=500, detail="VLM output format is invalid.")

        vlm_fraud_score = vlm_parsed_output.get("fraudScore", vlm_parsed_output.get("FraudScore", 0))
        # vlm_conf_score = vlm_parsed_output.get("confidenceScore", 0)
        vlm_explanation = vlm_parsed_output.get("explanation", vlm_parsed_output.get("Reason", "No explanation provided."))
        time_taken = time.time() - start_time
        # 2. VLM Integration and Thresholding
        vlm_is_reliable = False
        if request.category.lower() !="fake" :
            if (vlm_fraud_score < 0.25 or inner_dict.get("FraudScore", 0) <=0.25):
                vlm_is_reliable = True
            return JSONResponse(content={"finetuned_vlm_prediction": inner_dict,
                                      "fewshot_vlm_prediction": vlm_parsed_output,
                                      "finetuned_vlm_fraud_score": inner_dict.get("FraudScore", ""),
                                      "fewshot_fraud_score": vlm_fraud_score,
                                      "time_taken_finetuned": f"{time_taken_ft:.4f}",
                                         "time_taken_fewshot": f"{time_taken_ft:.4f}",
                                      "vlm_is_reliable": vlm_is_reliable})

        if request.category.lower()=="fake" and (vlm_fraud_score >= 0.75 or inner_dict.get("FraudScore", 0) >=0.75): # Define a threshold for VLM reliability
            vlm_is_reliable = True

        return JSONResponse(content={"finetuned_vlm_prediction": inner_dict,
                                     "fewshot_vlm_prediction": vlm_parsed_output,
                                     "finetuned_vlm_fraud_score": inner_dict.get("FraudScore", ""),
                                     "fewshot_fraud_score": vlm_fraud_score,
                                     "time_taken_finetuned": f"{time_taken_ft:.4f}",
                                         "time_taken_fewshot": f"{time_taken_ft:.4f}",
                                     "vlm_is_reliable": vlm_is_reliable})

    except HTTPException as http_exc:
        raise http_exc
    except Exception as e:
        print(f"An error occurred: {e}")
        traceback.print_exc()
        return JSONResponse(status_code=500, content={"error": f"An unexpected error occurred: {e}"})



In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn
# Headers to skip the ngrok browser warning
headers = {
    "ngrok-skip-browser-warning": "true"
}
auth_token = "placeholder"


# Set the authtoken
ngrok.set_auth_token(auth_token)
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://572ceb441bd4.ngrok-free.app


INFO:     Started server process [55728]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     106.222.230.74:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     106.222.230.74:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     106.222.230.74:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     106.222.230.74:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     106.222.230.74:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     106.222.230.74:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     106.222.230.74:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     106.222.230.74:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     106.222.230.74:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     106.222.230.74:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     106.222.230.74:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     106.222.230.74:0 - "POST /predict HTTP/1.1" 200 OK
